<a href="https://colab.research.google.com/github/Neharanjith/sciencefair10/blob/main/sciencefair10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as pt
import numpy as np
import pickle
import tensorflow as tf
from datetime import datetime, timedelta
!pip install streamlit_calendar
import keras
import seaborn as sns
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import Adagrad
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization,Bidirectional, GaussianNoise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install -q streamlit

In [ ]:
year07 = pd.read_csv('2007_CELIAS_Proton_Monitor_5min.csv')
year08 = pd.read_csv('2008_CELIAS_Proton_Monitor_5min.csv')
year09 = pd.read_csv('2009_CELIAS_Proton_Monitor_5min.csv')
year10 = pd.read_csv('2010_CELIAS_Proton_Monitor_5min.csv')
year11 = pd.read_csv('2011_CELIAS_Proton_Monitor_5min.csv')
year12 = pd.read_csv('2012_CELIAS_Proton_Monitor_5min.csv')
year13 = pd.read_csv('2013_CELIAS_Proton_Monitor_5min.csv')
year14 = pd.read_csv('2014_CELIAS_Proton_Monitor_5min.csv')
year15 = pd.read_csv('2015_CELIAS_Proton_Monitor_5min.csv')
year16 = pd.read_csv('2016_CELIAS_Proton_Monitor_5min.csv')
year17 = pd.read_csv('2017_CELIAS_Proton_Monitor_5min.csv')
year18 = pd.read_csv('2018_CELIAS_Proton_Monitor_5min.csv')
year19 = pd.read_csv('2019_CELIAS_Proton_Monitor_5min.csv')
year20 = pd.read_csv('2020_CELIAS_Proton_Monitor_5min.csv')
year21 = pd.read_csv('2021_CELIAS_Proton_Monitor_5min.csv')
year22 = pd.read_csv('2022_CELIAS_Proton_Monitor_5min.csv')
year23 = pd.read_csv('2023_CELIAS_Proton_Monitor_5min.csv')
dataset = pd.concat([year07, year08,year09,year10,year11,year12,year13,year14,year15,year16,year17,year18,year19,year20,year21,year22,year23], ignore_index=True)

In [ ]:
train_size = int(len(dataset) * .7) -5
val_size = int(len(dataset) * .15)
test_size = len(dataset) - train_size - val_size + 5
train = dataset.iloc[0:train_size, :]
val = dataset.iloc[train_size:train_size + val_size, :]
test = dataset.iloc[train_size + val_size:, :]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler_speed = MinMaxScaler(feature_range=(0, 1))
scaled_speed_train = scaler_speed.fit_transform(train[['SPEED']])
scaled_speed_val = scaler_speed.fit_transform(val[['SPEED']])
scaled_speed_test = scaler_speed.fit_transform(test[['SPEED']])


dataset['YY'] = dataset['YY'] + 2000
scaler_yearss = MinMaxScaler(feature_range=(0, 1))
scaled_yearss_train = scaler_yearss.fit_transform(train[['YY']])
scaled_yearss_val = scaler_yearss.fit_transform(val[['YY']])
scaled_yearss_test = scaler_yearss.fit_transform(test[['YY']])


scaler_nps = MinMaxScaler(feature_range=(0, 1))
scaled_nps_train = scaler_nps.fit_transform(train[['Np']])
scaled_nps_val = scaler_nps.fit_transform(val[['Np']])
scaled_nps_test = scaler_nps.fit_transform(test[['Np']])


scaler_n_ss = MinMaxScaler(feature_range=(0, 1))
scaled_n_ss_train = scaler_n_ss.fit_transform(train[['N/S']])
scaled_n_ss_val = scaler_n_ss.fit_transform(val[['N/S']])
scaled_n_ss_test = scaler_n_ss.fit_transform(test[['N/S']])


train['SPEED'] = scaled_speed_train
val['SPEED'] = scaled_speed_val
test['SPEED'] = scaled_speed_test

train['YY'] = scaled_yearss_train
val['YY'] = scaled_yearss_val
test['YY'] = scaled_yearss_test

train['Np'] = scaled_nps_train
val['Np'] = scaled_nps_val
test['Np'] = scaled_nps_test

train['N/S'] = scaled_n_ss_train
val['N/S'] = scaled_n_ss_val
test['N/S'] = scaled_n_ss_test


def encode_month(month):
    angle = (month - 1) / 11 * 2 * np.pi
    cos_value = np.cos(angle)
    return cos_value

days_month = 30
def encode_day(day, month):
    days_month = 30
    if month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12:
        days_month = 31
    elif month == 2:
      days_month = 28
    day_cos = np.cos(2 * np.pi * day / days_month)
    return day_cos


scaled_months_train = train['MON'].apply(encode_month)
scaled_months_test = test['MON'].apply(encode_month)
scaled_months_val = val['MON'].apply(encode_month)

scaled_days_train = train['DY'].combine(train['MON'], lambda day, month: encode_day(day, month))
scaled_days_test = test['DY'].combine(test['MON'], lambda day, month: encode_day(day, month))
scaled_days_val = val['DY'].combine(val['MON'], lambda day, month: encode_day(day, month))

train['MON'] = scaled_months_train
test['MON'] = scaled_months_test
val['MON'] = scaled_months_val

train['DY'] = scaled_days_train
test['DY'] = scaled_days_test
val['DY'] = scaled_days_val

train.drop(['DOY:HH:MM:SS','Vth','V_He','GSE_X','GSE_Y','GSE_Z','RANGE','CRN(E)','HGLAT','HGLONG'],axis=1,inplace=True)
test.drop(['DOY:HH:MM:SS','Vth','V_He','GSE_X','GSE_Y','GSE_Z','RANGE','CRN(E)','HGLAT','HGLONG'],axis=1,inplace=True)
val.drop(['DOY:HH:MM:SS','Vth','V_He','GSE_X','GSE_Y','GSE_Z','RANGE','CRN(E)','HGLAT','HGLONG'],axis=1,inplace=True)


<ipython-input-5-3031da7767d5>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['SPEED'] = scaled_speed_train
<ipython-input-5-3031da7767d5>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['SPEED'] = scaled_speed_val
<ipython-input-5-3031da7767d5>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [ ]:
def create_dataset(dataset, look_back=6):
    dataX = []
    dataY = []
    for i in range(len(dataset) - look_back):
        a = dataset.iloc[i: i + look_back, :].values
        dataX.append(a)
        b = dataset.iloc[i + 1: i + look_back + 1, :].values
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

look_back = 6

In [ ]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
valX, valY = create_dataset(val, look_back)

trainY = np.reshape(trainY,(int(trainY.shape[0]),6, 6))
testY = np.reshape(testY,(int(testY.shape[0]),6, 6))

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1, look_back)))
model.add((LSTM(units=65,return_sequences=True)))
model.add(BatchNormalization())
model.add(LSTM(units=65,return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=65,return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=65,return_sequences=True))
model.add(BatchNormalization())
model.add(GaussianNoise(0.01))
model.add(Dropout(0.0008))
model.add(Dense(6,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.compile(loss='mean_squared_error', optimizer= "Adam")
model.fit(trainX, trainY, epochs=10, batch_size=288, validation_data=None)

Epoch 1/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 357s 90ms/step - loss: 0.0932
Epoch 2/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 382s 90ms/step - loss: 0.0805
Epoch 3/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 377s 88ms/step - loss: 0.0798
Epoch 4/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 381s 88ms/step - loss: 0.0796
Epoch 5/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 381s 88ms/step - loss: 0.0796
Epoch 6/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 344s 88ms/step - loss: 0.0794
Epoch 7/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 381s 88ms/step - loss: 0.0794
Epoch 8/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 381s 88ms/step - loss: 0.0797
Epoch 9/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 384s 88ms/step - loss: 0.0794
Epoch 10/10
3884/3884 ━━━━━━━━━━━━━━━━━━━━ 343s 88ms/step - loss: 0.0794


In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
original_shapeY = trainY.shape
original_testY = testY.shape

trainY_reshaped = trainY.reshape(-1, original_shapeY[2])
testY_reshaped = testY.reshape(-1, original_testY[2])

scaler = MinMaxScaler(feature_range=(0, 2))
scaler.fit(trainY_reshaped)

trainPredict = scaler.inverse_transform(trainPredict.reshape(-1, trainPredict.shape[2]))
trainPredict = trainPredict.reshape(original_shapeY)

testPredict = scaler.inverse_transform(testPredict.reshape(-1, testPredict.shape[2]))
testPredict = testPredict.reshape(original_testY)

trainY = scaler.inverse_transform(trainY_reshaped)
trainY = trainY.reshape(-1, original_shapeY[1], original_shapeY[2])

testY = scaler.inverse_transform(testY_reshaped)
testY = testY.reshape(-1, original_testY[1], original_testY[2])

trainScore = np.sqrt(mean_squared_error(trainY.reshape(-1, trainY.shape[2]),trainPredict.reshape(-1, trainPredict.shape[2])))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY.reshape(-1, testY.shape[2]),testPredict.reshape(-1, testPredict.shape[2])))
print('Test Score: %.2f RMSE' % (testScore))

34954/34954 ━━━━━━━━━━━━━━━━━━━━ 225s 6ms/step
7491/7491 ━━━━━━━━━━━━━━━━━━━━ 51s 7ms/step
Train Score: 0.28 RMSE
Test Score: 0.27 RMSE


In [ ]:
predicted_values = []
middle = 1438098

testX = np.reshape(testX, (1, middle, 6))
current_input = testX.copy()

batch_size = 288
total_iterations = 261792
std_dev = 0.1

for batch_start in range(0, total_iterations, batch_size):
    batch_predictions = []  # Reset for each batch

    for _ in range(batch_size):
        last_time_step = current_input[:, -1:, :]  # Take the last known time step

        predicted_value = model.predict(last_time_step, verbose=0)

        # Apply stochastic sampling
        sampled_value = np.random.normal(loc=predicted_value[0], scale=std_dev)

        # Ensure correct shape
        sampled_value = np.reshape(sampled_value, (1, 1, current_input.shape[-1]))

        # Shift input window and insert the new sampled value
        current_input = np.roll(current_input, shift=-1, axis=1)
        current_input[0, -1, :] = sampled_value

        batch_predictions.append(sampled_value)

    # Append only the new batch's predictions
    predicted_values.extend(batch_predictions)

# Convert list to array
predicted_values = np.vstack(predicted_values).reshape(-1, 6)



predicted_values[:, 3] = scaler_speed.inverse_transform(predicted_values[:, 3].reshape(-1, 1)).flatten()
predicted_values[:, 0] = scaler_yearss.inverse_transform(predicted_values[:, 0].reshape(-1, 1)).flatten()
predicted_values[:, 4] = scaler_nps.inverse_transform(predicted_values[:, 4].reshape(-1, 1)).flatten()
predicted_values[:, 5] = scaler_n_ss.inverse_transform(predicted_values[:, 5].reshape(-1, 1)).flatten()

In [ ]:
speed_values = []
np_values = []
ns_values = []
final = []
newfinal = []
count = 0

# 0 is not visible, 1 is semi visible, 2 is very visible
for i, step in enumerate(predicted_values):
     final_speed = 0
     final_np = 0
     final_ns = 0
     if step[3] >= 400 and step[3] < 600:
       final_speed = 1
     elif step[3] >= 600:
       final_speed = 2
     if step[4] >= 20 and step[4] < 40:
       final_np = 1
     elif step[4] >= 40:
       final_np = 2
     if step[5] < 0:
       final_ns = 2
     sum = final_speed + final_np + final_ns
     if sum <= 2:
       final.append(("not visible"))
     elif sum <5:
       final.append(("semi visible"))
     else:
       final.append(("very visible"))

for i in range(0, len(final), 288):  # Process in chunks of 288
  count = 0
  for j in range(i, min(i + 288, len(final))):
    if j % 288 < 48 or j % 288 > 204:
        if final[j] == "semi visible":
            count += 1
        elif final[j] == "very visible":
            count += 2
  if count >= 100:
      newfinal.append("very visible")
  elif count >= 60:
      newfinal.append("semi visible")
  else:
      newfinal.append("not visible")

In [ ]:
import os

# ✅ Save `newfinal` to a text file and check if it was written correctly
file_path = "newfinal.txt"
with open(file_path, "w", encoding="utf-8") as f:
    for item in newfinal:
        f.write(item + "\n")

In [ ]:
%%writefile app.py

import streamlit as st
from streamlit_calendar import calendar
from datetime import datetime, timedelta
import pandas as pd
import json
import os

file_path = "newfinal.txt"
if os.path.exists(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
    newfinal = [line.strip() for line in f]
else:
  st.error("error")

st.set_page_config(page_title="CELESTIA", page_icon="🌌")

st.title("CELESTIA")

# Only allow 'multimonth' calendar mode
mode = "multimonth"

visibility_colors = {
    "very visible": "#9b3eed",
    "semi visible": "#b57de8",
    "not visible": "#d8bdf0",
}

# Start date (July 7, 2023)
start_date = datetime(2023, 7, 7)

# Generate events based on `newfinal` list (each index = a new day)
events = []
for i, visibility in enumerate(newfinal):
    event_start = start_date + timedelta(days=i)  # Each index corresponds to an exact date

    event = {
        "title": visibility.capitalize(),  # Convert to "Very Visible", "Semi Visible", etc.
        "color": visibility_colors[visibility],  # Assign the correct color
        "start": event_start.strftime("%Y-%m-%d"),
        "end": event_start.strftime("%Y-%m-%d"),  # Single-day event
        "resourceId": chr(97 + (i % 6)),  # Cycle through resource IDs (a-f)
    }
    events.append(event)

# Define calendar resources
calendar_resources = [
    {"id": "a", "building": "Building A", "title": "Room A"},
    {"id": "b", "building": "Building A", "title": "Room B"},
    {"id": "c", "building": "Building B", "title": "Room C"},
    {"id": "d", "building": "Building B", "title": "Room D"},
    {"id": "e", "building": "Building C", "title": "Room E"},
    {"id": "f", "building": "Building C", "title": "Room F"},
]

# Calendar options
calendar_options = {
    "editable": True,
    "navLinks": True,
    "resources": calendar_resources,
    "selectable": True,
    "initialView": "multiMonthYear",
}

st.markdown(
    """
    <style>
        .fc-day-today {
            background-color: #6632a8 !important;
            color: white !important;  /* Change text color if needed */
        }
        /* Change the color of the previous and next buttons (arrows) */
        .fc-prev-button, .fc-next-button {
            color: #6632a8 !important;  /* Set your desired arrow color */
            background-color: transparent !important;  /* Make the background transparent */
            border: none !important;  /* Remove borders if needed */
        }
    </style>
    """,
    unsafe_allow_html=True
)

# Render the calendar with updated event titles
state = calendar(
    events=st.session_state.get("events", events),
    options=calendar_options,
    custom_css="""
    .fc-event-past {
        opacity: 0.8;
    }
    .fc-event-time {
        font-style: italic;
    }
    .fc-event-title {
        font-weight: 700;
    }
    .fc-toolbar-title {
        font-size: 2rem;
    }
    """,
    key=mode,
)

# Store updated events in session state if changed
if state.get("eventsSet") is not None:
    st.session_state["events"] = state["eventsSet"]



Writing app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com